# 第2章 モデル、プロンプト、出力パーサー

 - [1. OpenAI API キーの設定](#1-openai-api-キーの設定)
 - [2. OpenAIの直接使用](#2-openaiの直接使用)
     - [2.1 1+1の計算](#21-11の計算)
     - [2.2 海賊メールをアメリカ英語で表現](#22-海賊メールをアメリカ英語で表現)
     - [2.3 日本語版](#23-日本語版)
 - [3. LangChain経由でのOpenAI使用](#3-langchain経由でのopenai使用)
     - [3.1 モデル](#31-モデル)
     - [3.2 プロンプトテンプレート](#32-プロンプトテンプレート)
         - [3.2.1 LangChainプロンプトテンプレートの使用](#321-langchainプロンプトテンプレートの使用)
         - [3.2.2 日本語版](#322-日本語版)
         - [3.2.3 プロンプトテンプレートが必要な理由](#323-プロンプトテンプレートが必要な理由)
     - [3.3 出力パーサー](#33-出力パーサー)
         - [3.3.1 出力パーサーがない場合](#331-出力パーサーがない場合)
         - [3.3.2 日本語版](#332-日本語版)
         - [3.3.3 LangChain出力パーサー](#333-langchain出力パーサー)
         - [3.3.4 日本語版](#334-日本語版)
 - [4. 補足資料](#4-補足資料)
     - [4.1 思考の連鎖推論（ReAct）](#41-思考の連鎖推論react)

## 1. OpenAI API キーの設定

[OpenAI アカウント](https://platform.openai.com/account/api-keys)にログインしてAPI キーを取得し、環境変数として設定してください。

- グローバル環境変数として設定したい場合は、[知乎文章](https://zhuanlan.zhihu.com/p/627665725)を参考にしてください。
- ローカル/プロジェクト環境変数として設定したい場合は、このファイルのディレクトリに`.env`ファイルを作成し、ファイルを開いて以下の内容を入力してください。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key" 
    </p>
  
  "your_api_key"をあなた自身のAPI キーに置き換えてください

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai.api_key = os.environ['OPENAI_API_KEY']  

## 2. OpenAIの直接使用

まず、OpenAIのAPIを直接呼び出すことから始めましょう。

`get_completion`関数は`openai`のラッパー関数で、与えられたプロンプト（prompt）に対して対応する回答を出力します。この関数は2つのパラメータを含みます
   
   - `prompt` 必須入力パラメータ。モデルに与える**プロンプトで、質問や、モデルに手伝ってもらいたいこと**（テキストの文体変更、翻訳、メッセージ返信など）が可能です。
   - `model` 非必須入力パラメータ。デフォルトでgpt-3.5-turboを使用します。他のモデルも選択できます。
   
ここでのプロンプトは、私たちがchatgptに与える質問に対応し、関数が出力する結果は、chatgptが私たちに与える回答に対応します。

In [2]:
from openai import OpenAI
def get_completion(prompt, model="gpt-3.5-turbo"):
    
    messages = [{"role": "user", "content": prompt}]
    client = OpenAI()

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message.content

### 2.1 1+1の計算

簡単な例から始めましょう - 日本語と英語でモデルに質問してみます

- 日本語プロンプト： `1+1は何ですか？`
- 英語プロンプト： `What is 1+1?`

In [ ]:
# 日本語
get_completion("1+1は何ですか？")

In [4]:
# 英文
get_completion("What is 1+1?")

'1+1 equals 2.'

### 2.2 海賊メールをアメリカ英語で表現

上記の簡単な例では、モデル`gpt-3.5-turbo`が1+1は何かという質問に回答しました。

今度はもう少し複雑な例を見てみましょう：

私たちがEコマース会社の従業員で、顧客が海賊Aだと仮定しましょう。彼は私たちのWebサイトでスムージーを作るためのブレンダーを購入しました。スムージーを作る過程で、スムージーの蓋が飛んでしまい、キッチンの壁が汚れてしまいました。そこで海賊Aは私たちのカスタマーサービスセンターに以下のメール`customer_email`を送りました：

In [5]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

私たちのカスタマーサービススタッフは、海賊の言い回しが少し理解しにくいと感じています。今、2つの小さな目標を実現したいと思います：

- モデルに海賊のメールをアメリカ英語の表現方法で翻訳してもらい、カスタマーサービススタッフがより理解しやすくする。*ここでの海賊の英語表現は英語の方言として理解でき、関西弁と標準語の関係のようなものです。
- モデルが翻訳する際に穏やかで敬意のあるトーンで表現してもらい、カスタマーサービススタッフの気分も良くする。

この2つの小さな目標に基づいて、テキスト表現スタイル：`style`を定義します

In [ ]:
# アメリカ英語 + 穏やか、敬意のあるトーン
style = """American English \
in a calm and respectful tone
"""

次にやるべきことは、`customer_email`と`style`を組み合わせて私たちのプロンプト:`prompt`を構築することです

In [ ]:
# 指定されたトーンに従って変換するようモデルに要求
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

`prompt`が構築できたので、`get_completion`を呼び出して私たちが望む結果を得ることができます - 穏やかで敬意のあるトーンのアメリカ英語で表現された海賊言葉のメール

In [8]:
response = get_completion(prompt)

In [9]:
response

"Ah, I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

言語スタイル変換の前後を比較すると、より正式な言葉遣いになり、極端な感情表現が置き換えられ、感謝の気持ちが表現されています。

✨ プロンプトを修正して、どんな異なる結果が得られるか試してみてください😉

### 2.3 日本語版

In [ ]:
# 標準日本語 + 穏やか、敬意のあるトーン
style = """正式な日本語 \
穏やかで敬意のあるトーンで
"""

In [ ]:
# 非正式な表現
customer_email = """   
ああ、腹が立つよ、\
ミキサーの蓋が飛んで、\
スムージーがキッチンの壁に飛び散ったんだ！\
もっと悪いことに、保証はキッチンの掃除費用をカバーしていない。\
今すぐ君の助けが必要だ、仲間よ！
"""

In [ ]:
# 指定されたトーンに従って変換するようモデルに要求
prompt = f"""三つのバッククォートで区切られたテキストを\
{style}スタイルに翻訳してください。
テキスト: ```{customer_email}```
"""

print(prompt)


In [13]:
response = get_completion(prompt)

response

'尊敬的先生/女士，\n\n我感到非常不安，因为我的搅拌机盖子掉了，导致奶昔溅到了厨房的墙上！更糟糕的是，保修不包括清洁厨房的费用。我现在需要您的帮助，朋友！感谢您的理解和支持。'

## 3. LangChain経由でのOpenAI使用

前の部分では、ラッパー関数`get_completion`を通じてOpenAIを直接呼び出し、方言メールの翻訳を完了し、穏やかで敬意のあるトーン、正式な日本語で表現されたメールを得ました。

LangChainを使用して同じ機能を実装してみましょう。

### 3.1 モデル

`langchain_community.chat_models.openai`からOpenAIの対話モデル`ChatOpenAI`をインポートします。OpenAI以外にも、`langchain_community.chat_models`は他の対話モデルも統合しています。詳細は[Langchain公式ドキュメント](https://python.langchain.com/en/latest/modules/models/chat/integrations.html)をご確認ください。

In [14]:
from langchain_community.chat_models.openai import ChatOpenAI

In [ ]:
# ここではtemperatureパラメータを0.0に設定し、生成される回答のランダム性を減らします。
# 毎回異なる新しいアイデアの回答を得たい場合は、このパラメータを調整してみてください。
chat = ChatOpenAI(temperature=0)
chat

上記の出力は、ChatOpenAIのデフォルトモデルが`gpt-3.5-turbo`であることを示しています

### 3.2 プロンプトテンプレート

前の例では、[f文字列](https://docs.python.org/ja/3/tutorial/inputoutput.html#tut-f-strings)を通じてPython表現の値`style`と`customer_email`を`prompt`文字列内に追加しました。

```python
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""
```
`langchain`はプロンプトを便利かつ迅速に構築・使用するためのインターフェースを提供します。今度は`langchain`を使用してプロンプトを構築する方法を見てみましょう。

#### 3.2.1 LangChainプロンプトテンプレートの使用

##### 1️⃣ プロンプトテンプレート文字列の構築
プロンプトテンプレート文字列：`template_string`を構築します

In [16]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

##### 2️⃣ LangChainプロンプトテンプレートの構築
`ChatPromptTemplate.from_template()`関数を呼び出して、上記のプロンプトテンプレート文字`template_string`をプロンプトテンプレート`prompt_template`に変換します

In [17]:
from langchain.prompts.chat import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [18]:
print(prompt_template.messages[0].prompt)

input_variables=['style', 'text'] template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'


上記の出力から、`prompt_template`には2つの入力変数があることがわかります：`style`と`text`。

In [19]:
print(prompt_template.messages[0].prompt.input_variables)

['style', 'text']


##### 3️⃣ テンプレートを使用して顧客メッセージプロンプトを取得

langchainプロンプトテンプレート`prompt_template`には2つの入力変数が必要です：`style`と`text`。ここではそれぞれ以下に対応します
- `customer_style`: 希望する顧客メールスタイル
- `customer_email`: 顧客の元のメールテキスト。

In [20]:
customer_style = """American English \
in a calm and respectful tone
"""

In [21]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

指定された`customer_style`と`customer_email`に対して、プロンプトテンプレート`prompt_template`の`format_messages`メソッドを使用して希望する顧客メッセージ`customer_messages`を生成できます。

In [22]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [23]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


`customer_messages`変数の型はリスト（`list`）で、リスト内の要素の変数型はlangchainカスタムメッセージ（`langchain.schema.HumanMessage`）であることがわかります。

最初の要素を印刷すると以下が得られます：

In [24]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [ ]:
# 日本語プロンプト
from langchain.prompts import ChatPromptTemplate

template_string = """三つのバッククォートで区切られたテキストを\
{style}スタイルに翻訳してください。\
テキスト: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

customer_style = """正式な日本語 \
穏やかで敬意のあるトーンで
"""

customer_email = """
ああ、腹が立つよ、\
ミキサーの蓋が飛んで、\
スムージーがキッチンの壁に飛び散ったんだ！\
もっと悪いことに、保証はキッチンの掃除費用をカバーしていない。\
今すぐ君の助けが必要だ、仲間よ！
"""

customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)


print(customer_messages[0])

##### 4️⃣ chatモデルを呼び出して顧客メッセージスタイルを変換

今度は[モデル](#model)部分で定義したchatモデルを呼び出して、顧客メッセージスタイルの変換を実現できます。これまでで、前の部分でのタスクを実装しました。

In [26]:
customer_response = chat.invoke(customer_messages, temperature=0.0)

In [27]:
print(customer_response.content)

非常抱歉听到您的困扰。我理解您的不快，因为搅拌机盖掉了，导致奶昔溅到了厨房的墙上。更令人不安的是，保修不包括清洁厨房的费用。现在我需要您的帮助，朋友。让我们一起解决这个问题。感谢您的理解和支持。


In [28]:
print(customer_response.content)

非常抱歉听到您的困扰。我理解您的不快，因为搅拌机盖掉了，导致奶昔溅到了厨房的墙上。更令人不安的是，保修不包括清洁厨房的费用。现在我需要您的帮助，朋友。让我们一起解决这个问题。感谢您的理解和支持。


##### 5️⃣ テンプレートを使用して返信メッセージプロンプトを取得

次に、さらに進んで、カスタマーサービススタッフの返信メッセージを海賊の言語スタイルに変換し、メッセージが比較的丁寧であることを確認します。

ここでは、2️⃣ステップで構築したlangchainプロンプトテンプレートを引き続き使用して、返信メッセージプロンプトを取得できます。

In [29]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [30]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [31]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

把由三个反引号分隔的文本翻译成一种a polite tone that speaks in English Pirate风格。文本: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



##### 6️⃣ chatモデルを呼び出して返信メッセージスタイルを変換

[モデル](#model)部分で定義したchatモデルを呼び出して返信メッセージスタイルを変換します

In [32]:
service_response = chat(service_messages)
print(service_response.content)

/Users/lta/anaconda3/envs/cookbook/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Ahoy there, me hearty customer! The warranty be not coverin' the cleanin' expenses for yer galley because 'tis yer own fault for misusin' yer blender by forgettin' to put the lid on afore startin' the blender. Tough luck, matey! Fare thee well!


#### 3.2.2 日本語版

In [ ]:
service_reply = """こんにちは、お客様、 \
保証は台所の清掃費用をカバーしません、 \
ブレンダーを始動する前に \
蓋をつけるのを忘れてブレンダーを誤用したためです。 \
これはあなたのミスです。 \
運が悪い！ さようなら！
"""

service_style_pirate = """\
丁寧な口調で \
正式な日本語を使って \
"""
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [34]:
service_response = chat(service_messages)
print(service_response.content)

尊敬的顾客，根据我们的保修政策，很遗憾告知您，厨房清洁费用不在保修范围内。这是因为在使用搅拌机之前，您忘记盖上盖子而导致搅拌机误用，这属于您的责任范围。非常抱歉给您带来不便。祝您好运，再见。


#### 3.2.3 プロンプトテンプレートが必要な理由

比較的複雑なシナリオに適用する場合、プロンプトは非常に長く、多くの詳細を含む可能性があります。**プロンプトテンプレートを使用することで、設計されたプロンプトをより便利に再利用できます**。

以下は比較的長いプロンプトテンプレートの例です。学生がオンライン学習を行い宿題を提出し、以下のプロンプトを通じて学生が提出した宿題の採点を実現します。

In [ ]:
# 英語版
prompt = """ Your task is to determine if the student's solution is correct or not

    To solve the problem do the following:
    - First, workout your own solution to the problem
    - Then compare your solution to the student's solution 
    and evaluate if the student's solution is correct or not.
    ...
    Use the following format:
    Question:
    ```
    question here
    ```
    Student's solution:
    ```
    student's solution here
    ```
    Actual solution:
    ```
    ...
    steps to work out the solution and your solution here
    ```
    Is the student's solution the same as actual solution \
    just calculated:
    ```
    yes or no
    ```
    Student grade
    ```
    correct or incorrect
    ```
    
    Question:
    ```
    {question}
    ```
    Student's solution:
    ```
    {student's solution}
    ```
    Actual solution:
    
    """

In [ ]:
# 日本語版
prompt = """ あなたのタスクは、学生の解答が正しいかどうかを判断することです

この問題を解決するために、以下を実行してください：
 - まず、問題に対するあなた自身の解決策を作成してください
 - 次に、あなたの解決策と学生の解決策を比較し
 学生の解決策が正しいかどうかを評価してください。
...
以下の形式を使用してください:

問題:
```
問題文
```
学生の解答:
```
学生の解答文
```
実際の解答:
```
...
解決策を作成する手順とあなたの解決策をここに参照してください
```
学生の解答と実際の解答が同じかどうか \
計算のみ：
```
はい または いいえ
```
学生の成績
```
正解 または 不正解
```

問題:
```
{question}
```
学生の解答:
```
{student's solution}
```
実際の解答:

"""

この他に、LangChainは一般的なシナリオ用のプロンプトテンプレートも提供しています。例えば、自動要約、質問応答、SQLデータベースへの接続、異なるAPIへの接続などです。LangChain内蔵のプロンプトテンプレートを使用することで、プロンプトの設計と構築に時間をかけることなく、素早く自分の大型モデルアプリケーションを構築できます。

最後に、大型モデルアプリケーションを構築する際、通常はモデルの出力が指定された形式になることを希望します。例えば、出力に特定のキーワードを使用して出力を構造化することです。以下は大型モデルを使用した思考の連鎖推論の例で、問題：*What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?*

LangChainライブラリ関数を使用することで、「Thought」（思考）、「Action」（行動）、「Observation」（観察）を思考の連鎖推論のキーワードとして使用し、出力を構造化できます。

```
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions

Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]

Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]
```

補足資料では、LangChainとOpenAIを使用した思考の連鎖推論の別のコード例をご覧いただけます。

### 3.3 出力パーサー

#### 3.3.1 出力パーサーがない場合

指定された評価`customer_review`に対して、情報を抽出し、以下の形式で出力したいと思います：

```python
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

In [37]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

##### 1️⃣ プロンプトテンプレート文字列の構築

In [38]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

##### 2️⃣ langchainプロンプトテンプレートの構築

In [39]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


##### 3️⃣ テンプレートを使用してプロンプトメッセージを取得

In [40]:
messages = prompt_template.format_messages(text=customer_review)

##### 4️⃣ chatモデルを呼び出して情報を抽出

In [41]:
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


##### 📝 分析と総括
`response.content`の型は文字列（`str`）であり、辞書（`dict`）ではないため、直接`get`メソッドを使用するとエラーが発生します。したがって、出力パーサーが必要です。

In [42]:
type(response.content)

str

In [43]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

#### 3.3.2 日本語版

In [ ]:
from langchain.prompts import ChatPromptTemplate

customer_review = """\
この葉っぱブロワーは非常に素晴らしいです。4つの設定があります：\
ろうそく消し、そよ風、風の街、竜巻。 \
2日で到着し、妻の\
記念日のプレゼントにちょうど間に合いました。 \
妻はとても気に入って言葉を失ったと思います。 \
今まで私だけが使用しており、\
一日おきの朝に芝生の葉っぱを掃除するのに使っています。 \
他の葉っぱブロワーより少し高価ですが、\
追加機能の価値があると思います。
"""

review_template = """\
以下のテキストから、以下の情報を抽出してください：

プレゼント：この商品は他の人へのプレゼントとして購入されましたか？ \
はいの場合は はい、いいえまたは不明の場合は いいえ と答えてください。

配送日数：商品が\
到着するまでに何日かかりましたか？ この情報が見つからない場合は-1を出力してください。

価格：価値や価格に関する文章を抽出し、\
それらをカンマ区切りのPythonリストとして出力してください。

以下のキーでJSONとして出力をフォーマットしてください：
プレゼント
配送日数
価格

テキスト: {text}
"""

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

In [45]:
messages = prompt_template.format_messages(text=customer_review)

chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
    "礼物": "是的",
    "交货天数": 2,
    "价钱": ["它比其他吹叶机稍微贵一点"]
}


#### 3.3.3 LangChain出力パーサー

##### 1️⃣ プロンプトテンプレート文字列の構築

In [46]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

##### 2️⃣ langchainプロンプトテンプレートの構築

In [47]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

##### 🔥 出力パーサーの構築

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")


response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

##### 3️⃣ テンプレートを使用してプロンプトメッセージを取得

In [49]:
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [50]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: 这款吹叶机非常神奇。 它有四个设置：吹蜡烛、微风、风城、龙卷风。 两天后就到了，正好赶上我妻子的周年纪念礼物。 我想我的妻子会喜欢它到说不出话来。 到目前为止，我是唯一一个使用它的人，而且我一直每隔一天早上用它来清理草坪上的叶子。 它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的。


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product     

##### 4️⃣ chatモデルを呼び出して情報を抽出

In [51]:
response = chat(messages)
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": "它比其他吹叶机稍微贵一点"
}
```


##### 5️⃣ 出力パーサーを使用して出力を解析

In [52]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': True, 'delivery_days': 2, 'price_value': '它比其他吹叶机稍微贵一点'}

##### 📝 分析と総括
`output_dict`の型は辞書（`dict`）で、直接`get`メソッドを使用できます。このような出力は下流タスクの処理により便利です。

In [53]:
type(output_dict)

dict

In [54]:
output_dict.get('delivery_days')

2

#### 3.3.4 日本語版

In [ ]:
# 日本語
review_template_2 = """\
以下のテキストから、以下の情報を抽出してください：

プレゼント：この商品は他の人へのプレゼントとして購入されましたか？
はいの場合は はい、いいえまたは不明の場合は いいえ と答えてください。

配送日数：商品が到着するまでに何日かかりましたか？ この情報が見つからない場合は-1を出力してください。

価格：価値や価格に関する文章を抽出し、それらをカンマ区切りのPythonリストとして出力してください。

テキスト: {text}

{format_instructions}
"""

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="プレゼント",
                             description="この商品は他の人へのプレゼントとして購入されましたか？\
                            はいの場合は はい、\
                            いいえまたは不明の場合は いいえ と答えてください。")

delivery_days_schema = ResponseSchema(name="配送日数",
                                      description="商品が到着するまでに何日かかりましたか？\
                                      この情報が見つからない場合は-1を出力してください。")

price_value_schema = ResponseSchema(name="価格",
                                    description="価値や価格に関する文章を抽出し、\
                                    それらをカンマ区切りのPythonリストとして出力してください")


response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [56]:
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: 这款吹叶机非常神奇。 它有四个设置：吹蜡烛、微风、风城、龙卷风。 两天后就到了，正好赶上我妻子的周年纪念礼物。 我想我的妻子会喜欢它到说不出话来。 到目前为止，我是唯一一个使用它的人，而且我一直每隔一天早上用它来清理草坪上的叶子。 它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的。


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？                            如果是，则回答 是的，                            如果否或未知，则回答 不是。
	"交货天数": string  // 产品需要多少天才能到达？                                      如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，                                    并将它们输出为逗号分隔的 Python 列表
}
```



In [57]:
response = chat(messages)
print(response.content)


```json
{
	"礼物": "是的",
	"交货天数": "两天",
	"价钱": "它比其他吹叶机稍微贵一点"
}
```


In [58]:
output_dict = output_parser.parse(response.content)
output_dict

{'礼物': '是的', '交货天数': '两天', '价钱': '它比其他吹叶机稍微贵一点'}

## 4. 補足資料

### 4.1 思考の連鎖推論（ReAct）
参考資料：[ReAct (Reason+Act) prompting in OpenAI GPT and LangChain](https://tsmatz.wordpress.com/2023/03/07/react-with-openai-gpt-and-langchain/)

In [ ]:
from langchain.docstore.wikipedia import Wikipedia
from langchain_community.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="Search for a term in the docstore.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="Lookup a term in the docstore.",
  )
]

# 大規模言語モデルを使用
llm = ChatOpenAI(
  model_name="gpt-3.5-turbo",
  temperature=0,
)

# ReActエージェントを初期化
react = initialize_agent(tools, llm, agent="react-docstore", verbose=True)
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=react.agent,
  tools=tools,
  verbose=True,
)


question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
agent_executor.run(question)